# **Note de synthèse**

# Résumé

Cette note présente le processus de modélisation et d'interprétation du modèle développé.

Home Credit (société de crédit à la consommation) souhaite construire un système de scoring automatique qui prédit la probabilité de défaut d'un client qui effectue une demande de prêt.

Il s'agit donc d'un problème de classification binaire supervisée (la variable cible (`TARGET`) prendra la valeur 1 en cas de risque de défaut client, et 0 sinon).

# Préparation des données

Comme dans tout projet de **prédiction automatique** basée sur des données existantes, la part la plus importante du travail du data scientist porte sur le déterminant principal de la qualité du modèle prédictif à construire : la qualité des données qui lui sont fournies en entrée pour l'entraîner.

Le **préparation des données**, ou **pré-traitement**, consiste en les étapes principales suivantes :
* l'**analyse exploratoire**,
* l'**ingénierie des caractéristiques**.

## Analyse exploratoire

Il s'agit de comprendre les données, d'abord en termes de dimensions, de densité d'information, de taux de remplissage, de qualité

Il s'agit ensuite de les comprendre en termes métiers, sur la base de la documentation et des déductions que l'on peut faire sur leur utilité et leur emploi par rapport aux enjeux stratégiques de l'entreprise.

Il s'agit enfin de les réduire en termes de caractéristiques statistiques (tendance centrale, dispersion, etc) interprétables et utilisables tant d'un point de vue métier que technique.

En particulier, dans le contexte d'un projet d'apprentissage supervisé d'un classifieur, il est utile, dès l'analyse exploratoire, de s'interroger sur la corrélation entre chaque variables et les classes cibles. Cela permet notamment d'anticiper prudemment sur leur importance respective dans le modèle à construire, c'est-à-dire leur poids relatif en tant que facteurs déterminants de la prédiction.

## Ingénierie des caractéristiques

En s'appuyant sur la connaissance intime des données dégagée par l'analyse exploratoire, l'ingénieur data peut élaborer ses stratégies pour *nettoyer les données*, c'est-à-dire pour augmenter leur qualité. Il s'agit d'identifier les outliers et d'en comprendre la nature, soit d'exception bien réelle à la norme, soit de donnée aberrante produite par une défaillance dans l'alimentation (saisie, capteurs), puis soit de les corriger, soit de les éliminer pour éviter qu'ils ne biaisent le modèle à construire. Il s'agit, dans la mesure du possible, de compléter les valeurs manquantes sur la base de techniques d'interpolation simples. Il s'agit parfois de *parser* des données semi-structurées pour en faire ressortir la structure et la décomposition en plusieurs données plus élémentaires. Il s'agit enfin, en lien avec le métier, de dériver des caractéristiques (variables) supplémentaires pertinentes calculées à partir de celles qui sont données, comme des rapports ou des produits de variables élémentaires.

# Entraînement du modèle

## Les étapes avant l'entraînement des modèles

L’étape d’entraînement des modèles est précédée par 4 étapes principales suivantes : 
1. ***Feature engineering*** : la création des features pertinents pour la modélisation a été entièrement inspirée du notebook Kaggle suivant https://www.kaggle.com/jsaguiar/lightgbmwith-simple-features/.
2. **Sampling** : Le jeu de données étant relativement important, nous avons utilisé un sampling des données (100000 clients).
3. **Séparation des données** : Séparation train (80 %) et test (20 %) des données nous sauvegardons en csv les données test qui seront nos clients d'exemples pour API et dashboard. 
4. **Pré-traitement des données** : Imputation des valeurs manquantes (via un SimpleImputer [strategy=median] de la librairie Scikit-Learn) et à normaliser les valeurs (via un Min-MaxScaler de la librairie Scikit-Learn qui ramène l'ensemble des valeurs entre 0 et 1).

## Rééquilibrage des classes

Nous sommes dans ce cas confrontés à un problème de classification binaire d’une base de données avec des classes déséquilibrée :
* environ 92 % des données sont des clients à qui on accorde le prêt (classe 0 majoritaire).
* environ 8 % des données sont des clients à qui on refuse le prêt (classe 1 minoritaire).

La bibliothèque `imblearn` offre des fonctionnalités qui adressent ce problème :
- Undersampling (RandomUnderSampler de la librairie Imbalanced-Learn) : suppression des observations de la classe majoritaire afin de rééquilibrer le jeu de données.
- Oversampling (SMOTE de la librairie Imbalanced-Learn) : répétition des observations de la classe minoritaire afin de rééquilibrer le jeu de données.
- Class_weight="balanced" : argument qui indique le déséquilibre à l’algorithme afin qu'il en tienne compte directement.

## Entraînement des modèles et choix du meilleur modèle

Afin de modéliser au mieux notre problème, nous allons comparer les performances de 3 algorithmes :
* **Régression Logistique** : (famille des algorithmes linéaires) via la classe LogisticRegression de la librairie Scikit-Learn
* **RandomForestClassifier** : (famille des algorithmes ensemblistes) via la classe RandomForestClassifier de la librairie Scikit-Learn
* **Light Gradient Boosting Machine** : (famille des algorithmes gradient boosting) via la classe LGBMClassifier de la librairie lightgbm.

Leurs performances ont été comparées à celles d'une baseline naïve, de la classe `DummyClassifier`, instanciée avec la stratégie `"most_frequent"`, c’est-à-dire prédisant systématiquement la classe la plus fréquente. La métrique utilisée pour évaluer les performances de chacun des modèles est l'AUC (Area Under the ROC Curve). Plus l'AUC est élevée, plus le modèle est capable de prédire les 0 comme 0 et les 1 comme 1.

Tous les modèles ont été entraînés avec une validation croisée (cv=5) et leurs hyperparamètres ont été optimisés à l'aide d'une recherche en grille (`GridSearch`). Les algorithmes ont été évalués avec chacune des méthodes de rééquilibrage des classes présentées plus haut. 

Le meilleur modèle a été choisi sur la base de la valeur du score AUC moyen obtenu sur l'ensemble des splits de la validation croisée (`GridSearchCv`). Le score AUC obtenu sur le jeu de test a également été calculé. L’algorithme LGBM associé à la stratégie de rééquilibrage consistant à indiquer `"balanced"` comme valeur pour `"class_weight"` donne les meilleurs résultats.

# Fonction coût et algorithme d'optimisation

## Fonction coût

### Métrique d’évaluation

La matrice de confusion est la suivante :

**Tableau ou image**
* Classe prédite
    * Classe 0 : non défaillant
    * Classe 1 : défaillant
* Classe réelle
    * Classe 0 : non défaillant
        * Vrais négatifs TN
        * Faux positifs FP
    * Classe 1 : défaillant
        * Faux négatifs FN
        * Vrais positifs TP

* **TP (Vrais Positifs)** : la classe a été prédite comme “1”, et la classe réelle est bien “1” => le modèle a prédit que le client ne rembourserait pas, et il a bien été en défaut.
* **TN (Vrais Négatifs)** : la classe a été prédite comme “0”, et la classe réelle est bien “0”=> le modèle a prédit que le client rembourserait, et il l’a bien fait.
* **FP (Faux Positifs)** : la classe a été prédite comme “1”, mais la classe réelle est “0” => le modèle a prédit que le client ne rembourserait pas, mais il a bien remboursé son crédit.
* **FN (Faux Négatifs)** : la classe a été prédite comme “0”, mais la classe réelle est “1” => le modèle a prédit que le client rembourserait, mais il a été en défaut.

La problématique « métier » est de prendre en compte qu'un faux positif (bon client considéré comme mauvais = crédit non accordé à tort, donc manque à gagner de la marge pour la banque) n'a pas le même coût qu'un faux négatif (mauvais client à qui on accorde un prêt, donc perte sur le capital non remboursé). Un faux négatif est en effet 10 fois plus coûteux qu'un faux positif. Nous avons ainsi défini une fonction coût métier adaptée au projet qui permet d'attribuer plus de poids à la minimisation des FN.

Des coefficients permettant de pondérer les pénalités associées à chaque cas de figure ont été attribués :

TP_value : 0 ; FN_value : -10 ; TN_value : 1 ; FP_value : -1


Ces valeurs de coefficients signifient que les Faux Négatifs engendrent des pertes 10 fois plus importantes que les pertes engendrées par les Faux Positifs. Les Vrais positifs n'entrainent aucun gain ni perte pour l'entreprise, tandis que les Vrais Négatifs permettent à l’entreprise de gagner en prêtant à de bons prospects. Ces poids ont été fixé arbitrairement et il est tout à fait envisageable de les modifier à la convenance de l’optique métier.

# Algorithme d’optimisation

Une fois le modèle choisi (LGBM) et ses hyperparamètres optimisés d’un point de vue technique (via l’AUC avec GridSearch), nous avons effectué une nouvelle recherche des hyperparamètres via HyperOpt (algorithme qui utilise l’optimisation bayésienne) en nous basant sur la fonction coût métier proposée. De cette manière, les hyperparamètres optimaux du modèle sont choisis de sorte à minimiser la perte pour l'entreprise. HyperOpt nécessite 4 paramètres pour une implémentation de base à savoir : la fonction objectif (à minimiser loss = 1-score), l'espace de recherche (plages pour les hyperparamètres), l'algorithme d'optimisation et le nombre d'itérations.

# Interprétabilité globale et locale du modèle

Le dashboard est destiné à aux agents de Home Credit qui doivent mettre en oeuvre une politique de transparence.

Ces derniers doivent être en mesure de motiver l'éventuelle suite non favorable donnée à une demande de prêt.

Pour cela, la méthode SHAP a été utilisée afin de comprendre comment chacune des variables du client à influencer sur la décision finale du modèle.

## Feature importances globale

Sur l'image de gauche, l'importance des variables est calculée en moyennant la valeur absolue des valeurs de Shap. Sur la droite, les valeurs de Shap sont représentées pour chaque variable dans leur ordre d’importance, chaque point représente une valeur de Shap, les points rouges représentent des valeurs élevées de la variable et les points bleus des valeurs basses de la variable.

**IMAGE**

## Feature importances locale

La méthode SHAP calcule les shap_values : l'impact d’une variable (sur la prédiction) pour chaque ligne des données. 

Les valeurs SHAP sont additives : les valeurs en rouge augment la valeur prédit (risque d’être défaillant), une valeur bleu réduit la prédiction (basse du risque d’être défaillant).

**IMAGE**

# API Flask

Une API Flask qui permet de prédire le score (pourcentage de risque de défaut) d’un client à partir de son identifiant.

# Axes d’amélioration

* Après le feature engineering inspiré du Kaggle mentionné plus haut, les données contiennent 798 features au total. Pour la modélisation, nous aurions pu ajouter une étape de sélection des features afin de ne retenir que les features les plus importants et réduire ce nombre.
* La fonction coût métier définie utilise des poids de pénalités arbitrairement fixés. Il est donc certainement possible d’affiner les prédictions du modèle en fixant ces poids de façon plus adéquate par des experts métier.